In [14]:
from email.header import decode_header
import re
from imap_tools import MailBox
from imap_tools import AND
import pandas as pd
import dateparser
from dateparser.search import search_dates

In [31]:
from imap_tools import MailboxLoginError

password = "7PP4ZDXqnv"
username = "lazutkina@cs.vsu.ru"
host = "info.vsu.ru"

mailbox = MailBox(host)
mailbox.login(username=username, password=password, initial_folder='INBOX')
# try:
#     res = mailbox.login(username=username, password=password, initial_folder='INBOX')
#     print(res.login_result[0])
# except MailboxLoginError as e: 
#     print(e.command_result)
# except Exception as e:
#     print(e.command_result)


In [35]:
mails = mailbox.fetch(limit=1, reverse=True, mark_seen=False,)

In [37]:
for msg in mails:
    print(msg.uid, "\n", msg.subject, "\n",  msg.text)
    

In [4]:
mailbox.select("inbox")
result, data = mailbox.search(None, "ALL")
id_list = data[0].split()
latest_email_id = id_list[-1]

AttributeError: 'MailBox' object has no attribute 'select'

In [20]:
def get_folders_list(imap):
    folders = []
    for i in imap.folder.list('INBOX'):
        k = i.name.split('/')
        folders.append(k[1])
    return folders

In [21]:
get_folders_list(mailbox)

['Заявки', 'К пересылке', 'Старые', 'тест']

In [15]:
def mails_to_df(mails):
    columns = ['id', 'subject', 'from', 'text', 'date', 'attachment_filename', 'attachment_content_type', 'headers']
    mails_list = []
    for msg in mails:
        attachment_filename = ",".join(str(att.filename) for att in msg.attachments)
        attachment_content_type = ",".join(str(att.content_type) for att in msg.attachments)
        
        mails_list.append([msg.uid, msg.subject, msg.from_, msg.text, msg.date_str, attachment_filename, attachment_content_type, msg.headers])

    df = pd.DataFrame(data=mails_list)
    df.columns = columns
    return df

## Подстановка правильных данных

In [16]:
source_df = mails_to_df(mails)
source_df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,"Thu, 16 May 2024 11:53:29 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,"Thu, 16 May 2024 11:53:32 +0300",Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,"Thu, 16 May 2024 11:53:29 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,"Thu, 16 May 2024 11:53:30 +0300",Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,"Thu, 16 May 2024 11:53:30 +0300",рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
1132,1213,FWD:Re: Заседание редколлегии,cs-it@cs.vsu.ru,"Вероника, привет!\r\nСреда - единственный день...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1133,1214,FWD:Re[2]: сверка статьи,cs-it@cs.vsu.ru,"Вероника Викторовна, на сайте уже появился 2 н...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1134,1215,FWD:Re: статья на рецензирование,cs-it@cs.vsu.ru,"Уважаемая Вероника Викторовна,\r\nпредставленн...","Thu, 16 May 2024 12:06:45 +0300",,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1135,1216,FWD:,cs-it@cs.vsu.ru,Уважаемые Коллеги!\n\nПрошу Вас рассмотреть мо...,"Thu, 16 May 2024 12:06:45 +0300",4прост. фрактал 22342+.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [17]:
source_df.to_csv("files/1_source_mails.csv", index=False)

In [18]:
df = source_df.copy()

In [19]:
def preprocess_date(df):
    for index, row in df.iterrows():
        headers = row['headers']
        text = row['text']
        date = row['date']
        if 'x-original-date' in headers:
            date = headers['x-original-date'][0]
            
        if 'Sent:' in text:
            date = re.search(r'Sent: (.*?)\n', text).group(1)
            
        dp = dateparser.parse(date)
        if dp is not None:
            date = dp.strftime("%d.%m.%Y %H:%M:%S")
        else:
            date = search_dates(date)[0][1].strftime("%d.%m.%Y %H:%M:%S")
            
        df.at[index, 'date'] = date
    return df

In [20]:
df = preprocess_date(df)
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [21]:
fixed_dates_df = df.copy()

In [22]:
df = fixed_dates_df.copy()
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,cs-it@cs.vsu.ru,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,cs-it@cs.vsu.ru,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,cs-it@cs.vsu.ru,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,cs-it@cs.vsu.ru,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [23]:
def preprocess_from(df):
    for index, row in df.iterrows():
        headers = row['headers']
        text = row['text']
        x_from = row['from']
        if 'x-original-from' in headers:
            x_from = headers['x-original-from'][0]
        elif 'From:' in text:
            x_from = re.search(r'From: (.*?)\n', text).group(1)
            
        if len(decode_header(x_from)) == 2:
            encoding = decode_header(x_from)[0][1]
            name = decode_header(x_from)[0][0]
            if encoding is not None:
                name = name.decode(encoding)
            
            address = decode_header(x_from)[1][0].decode()
            x_from = name + ' ' + address
            
        df.at[index, 'from'] = x_from
    return df

In [24]:
preprocess_from(df)

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,Павловский Максим Викторович <pmv-160570@yande...,\r\n\r\n-----Original Message-----\r\nFrom: Па...,10.06.2018 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,Михаил Горчаков <gorchakovmihail@mail.ru> \r,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,03.06.2018 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,Аснина Н.Г. <andrey050569@yandex.ru> \r,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,11.06.2018 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,Матвеев Михаил <mgmatveev@yandex.ru> \r,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,08.06.2018 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,Alexander Krylovetsky <aakryl@sc.vsu.ru> \r,\r\n\r\n-----Original Message-----\r\nFrom: Al...,07.06.2018 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
1132,1213,FWD:Re: Заседание редколлегии,Irina Voronina <irina.voronina@gmail.com>,"Вероника, привет!\r\nСреда - единственный день...",04.09.2018 16:11:03,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1133,1214,FWD:Re[2]: сверка статьи,сергеев александр <sergeev00765@mail.ru>,"Вероника Викторовна, на сайте уже появился 2 н...",05.09.2018 09:01:06,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1134,1215,FWD:Re: статья на рецензирование,"""Sergey A. Zapryagaev"" <zsa@cs.vsu.ru>","Уважаемая Вероника Викторовна,\r\nпредставленн...",06.09.2018 15:35:28,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1135,1216,FWD:,Александр Щербаков <a.scherbakov58@yahoo.com>,Уважаемые Коллеги!\n\nПрошу Вас рассмотреть мо...,21.04.2018 00:00:00,4прост. фрактал 22342+.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [25]:
fixed_from_df = df.copy()

In [26]:
df['date'] = pd.to_datetime(df['date'], format="%d.%m.%Y %H:%M:%S")
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,Павловский Максим Викторович <pmv-160570@yande...,\r\n\r\n-----Original Message-----\r\nFrom: Па...,2018-06-10 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,Михаил Горчаков <gorchakovmihail@mail.ru> \r,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,2018-06-03 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,Аснина Н.Г. <andrey050569@yandex.ru> \r,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,2018-06-11 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,Матвеев Михаил <mgmatveev@yandex.ru> \r,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,2018-06-08 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,Alexander Krylovetsky <aakryl@sc.vsu.ru> \r,\r\n\r\n-----Original Message-----\r\nFrom: Al...,2018-06-07 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [27]:
def preprocess_subject(subjects: list[str]):
    column = []
    for s in subjects:
        if s:
            s = s.removeprefix("FWD:").removeprefix("Fwd:")
            column.append(s)
        else:
            column.append('Нет темы')
    return column


In [28]:
df['subject'] = preprocess_subject(df['subject'])
df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,70,FW: Вестник САИТ 1-2018,Павловский Максим Викторович <pmv-160570@yande...,\r\n\r\n-----Original Message-----\r\nFrom: Па...,2018-06-10 10:01:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,71,FW: Проверка статьи,Михаил Горчаков <gorchakovmihail@mail.ru> \r,\r\n \r\nFrom: Михаил Горчаков <gorchakovmiha...,2018-06-03 23:57:00,Исправления_Горчаков.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,72,FW: e-mail Баевой Н.Б.,Аснина Н.Г. <andrey050569@yandex.ru> \r,\r\n \r\nFrom: Аснина Н.Г. <andrey050569@yand...,2018-06-11 11:38:00,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,73,FW: от Матвеева,Матвеев Михаил <mgmatveev@yandex.ru> \r,\r\n \r\nFrom: Матвеев Михаил <mgmatveev@yand...,2018-06-08 08:49:00,Рецензия.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,74,FW: рецензия,Alexander Krylovetsky <aakryl@sc.vsu.ru> \r,\r\n\r\n-----Original Message-----\r\nFrom: Al...,2018-06-07 00:57:00,рецензия_Шмырин.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
1132,1213,Re: Заседание редколлегии,Irina Voronina <irina.voronina@gmail.com>,"Вероника, привет!\r\nСреда - единственный день...",2018-09-04 16:11:03,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1133,1214,Re[2]: сверка статьи,сергеев александр <sergeev00765@mail.ru>,"Вероника Викторовна, на сайте уже появился 2 н...",2018-09-05 09:01:06,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1134,1215,Re: статья на рецензирование,"""Sergey A. Zapryagaev"" <zsa@cs.vsu.ru>","Уважаемая Вероника Викторовна,\r\nпредставленн...",2018-09-06 15:35:28,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1135,1216,,Александр Щербаков <a.scherbakov58@yahoo.com>,Уважаемые Коллеги!\n\nПрошу Вас рассмотреть мо...,2018-04-21 00:00:00,4прост. фрактал 22342+.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [29]:
# Подставлены правильные данные, но текст "грязный"
df.to_csv('files/2_fixed_values.csv', index=False)

In [30]:
df = df.sort_values(by='date')
df.head()

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
11,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
12,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,,,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
19,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
27,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
21,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [31]:
# Данные отсортированы по дате, текст все еще грязный
df.to_csv('files/3_sorted_df.csv', index=False)

In [32]:
df = pd.read_csv('files/3_sorted_df.csv')

In [33]:
df.shape

(1137, 8)

In [34]:
df = df.drop_duplicates('text')

In [35]:
df.shape

(825, 8)

In [36]:
# Данные отсортированы по дате и удалены дубликаты, текст грязный
df.to_csv('files/4_drop_duplicates_df.csv', index=False)

### Разметка

In [ ]:
categories = ['submitting an article', 'checking an article', 'revision', 'ready for review', 'reviewing', 'rejected', 'publication']
categories_rus = ['подача статьи', 'проверка статьи', 'доработка', 'готово к рецензированию', 'рецензирование', 'отклонена', 'публикация']

In [ ]:
submitting_keywords = 'статья в вестник вгу, статья для, статья для публикации, статья на публикацию, публикация статьи, статья, публикация, статья вак'
ready_for_review_keywords = 'статья для рецензирования, статья экспертное заключение'

In [ ]:
# def fill_category(dataframe: pd.DataFrame):
#     col = []
#     for i, row in dataframe.iterrows():
        

In [228]:
new_df = pd.read_csv('files/drop_duplicates_df.csv')
new_df

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
...,...,...,...,...,...,...,...,...
820,1065,Re: статья на рецензию,Алексей Буховец <abuhovets@mail.ru>,Высылаю рецензию на материал А.М. Сагдатуллина...,2019-11-14 09:40:50,Рецензии Сагдатуллина А_М_САИТ.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
821,1068,статья,hein minzaw <heinminnzaw13@gmail.com>,Здравствуйте !\r\nМою отправлен нучную статью ...,2019-11-14 14:40:59,"Рис1.docx,Рис2.docx,Статья .doc",application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
822,1069,Re: Королев= ответы на замечания рецензента,Сергей Медведев <s_n_medvedev@mail.ru>,"Здравствуйте.\nСчитаю, что статья может быть о...",2019-11-15 00:03:13,"Ответ после повторного рецензирования.pdf,Реце...","application/pdf,application/vnd.openxmlformats...",{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
823,1066,Re: Конденсированные среды,CONDENSED MATTER AND INTERPHASES <kcmf@main.vs...,"Добрый день, Виктория Викторовна!\r\n\r\nЮрист...",2019-11-15 09:43:16,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [229]:
df['subject'].to_csv('files/subjects.csv', index=False)

In [242]:
subjects = df[['id', 'subject']]
subjects

,id,subject
11,82,Повторная отправка
12,83,Re: Шаблон рецензии (новый)
19,90,RE: статья Вестник САИТ Рецензия
27,98,RE: статья для рецензирования
21,92,RE: рецензия на статью
...,...,...
987,1065,Re: статья на рецензию
990,1068,статья
991,1069,Re: Королев= ответы на замечания рецензента
988,1066,Re: Конденсированные среды


In [243]:
subjects['subject'] = clean_text(subjects['subject'])

C:\Users\08642\AppData\Local\Temp\ipykernel_22400\4018698362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subjects['subject'] = clean_subject(subjects['subject'])


In [246]:
uniq_sbj = subjects.drop_duplicates(subset='subject')
uniq_sbj

,id,subject
11,82,повторная отправка
12,83,шаблон рецензии новый
19,90,статья вестник саит рецензия
27,98,статья для рецензирования
21,92,рецензия на статью
...,...,...
1000,1079,сведения об авторах doc статья кротов docx экс...
986,1064,статья колегова о а
987,1065,статья на рецензию
991,1069,королев ответы на замечания рецензента


In [247]:
uniq_sbj.to_csv('files/unique_subjects.csv', index=False)

## Ярги парсер

In [76]:
test_df = pd.read_csv('files/drop_duplicates_df.csv')
test_df.head() 

,id,subject,from,text,date,attachment_filename,attachment_content_type,headers
0,82,Повторная отправка,Павловский Максим Викторович <pmv-160570@yand...,Добрый вечер!\r\nОтсылаю материалы рецензии на...,2016-01-14 20:47:03,Рецензия Гаршиной.doc,application/msword,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
1,83,Re: Шаблон рецензии (новый),Юрий Нечаев <nechaev_ub@mail.ru>,Спасибо.Нечаев.,2018-02-04 19:56:26,NaN,NaN,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
2,90,RE: статья Вестник САИТ Рецензия,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-09 17:36:00,Рецензия Толстоброва на статью А.С. Платонова....,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
3,98,RE: статья для рецензирования,Вялых Сергей Ариевич <vyalyh@govvrn.ru>,"Доброе время суток, Вероника Викторовна!\r\n\r...",2018-02-09 17:54:00,рецензия_на_Проблема защита_пользователей.docx,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...
4,92,RE: рецензия на статью,Толстобров А.П. <tap@vsu.ru>,Добрый день!\r\nПосылаю рецензию\r\n\r\nС уваж...,2018-02-11 22:37:00,Рецензия Толстоброва на статью Черненькая-Маге...,application/vnd.openxmlformats-officedocument....,{'dkim-signature': ('v=1; a=rsa-sha256; q=dns/...


In [36]:
text = test_df.iloc[20]['text']
text

'Здравствуйте!\r\n\r\nПишет Вам аспирант математического факультета ВГУ Литвинов Дмитрий!\r\nЯ собираюсь отправить свою статью в журнал "Вестник ВГУ. Серия: Системный анализ и информационные технологии". В связи с этим у меня есть несколько вопросов:\r\n\r\n\r\n1)Принимаете ли Вы статьи аспирантов?\r\n2)Какие документы и прочие материалы я должен предоставить вместе со статьей в редакцию?\r\n3)Можно ли предоставить электронную версию статьи в формате tex?\r\n4)Как долго происходит процесс рецензирования в журнале?\r\n\r\nЗаранее спасибо!\r\n'

In [32]:
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import gram, dictionary, type as tp
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import gram,  eq, normalized
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from yargy.tokenizer import RULES
from ipymarkup import show_span_box_markup as show_markup

In [48]:
TOKENIZER = MorphTokenizer()
list(TOKENIZER('поинтересуюсь'))

[MorphToken(
     value='поинтересуюсь',
     span=[0, 13),
     type='RU',
     forms=[Form('поинтересоваться', Grams(1per,VERB,futr,indc,intr,perf,sing))]
 )]

In [82]:
subjects = test_df[['id', 'subject', 'attachment_filename']]
subjects

,id,subject,attachment_filename
0,82,Повторная отправка,Рецензия Гаршиной.doc
1,83,Re: Шаблон рецензии (новый),NaN
2,90,RE: статья Вестник САИТ Рецензия,Рецензия Толстоброва на статью А.С. Платонова....
3,98,RE: статья для рецензирования,рецензия_на_Проблема защита_пользователей.docx
4,92,RE: рецензия на статью,Рецензия Толстоброва на статью Черненькая-Маге...
...,...,...,...
820,1065,Re: статья на рецензию,Рецензии Сагдатуллина А_М_САИТ.docx
821,1068,статья,"Рис1.docx,Рис2.docx,Статья .doc"
822,1069,Re: Королев= ответы на замечания рецензента,"Ответ после повторного рецензирования.pdf,Реце..."
823,1066,Re: Конденсированные среды,NaN


In [83]:
subjects['category'] = ''


C:\Users\08642\AppData\Local\Temp\ipykernel_10472\2351832943.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subjects['category'] = ''


In [117]:
Category = fact('Category', ['name'])
article = morph_pipeline(
    ['опубликовать','публикация','опубликования','разместить',
        'материалы','размещение','печатать','напечатать','принять']
).interpretation(Category.name.const('подача'))

review = morph_pipeline(
    ['рецензия на статью','рецензия','отрецензировать',
        'рецензирование','рецензент']
).interpretation(Category.name.const('рецензия'))

question = morph_pipeline(
    ['вопрос','подсказать','сказать','?','узнать',
        'интересоваться','поинтересоваться', 'интересует','повод']
).interpretation(Category.name.const('вопрос'))

ARTICLE = rule(
    or_(article, review, question)).interpretation(Category)
parser = Parser(ARTICLE)
# matches = list(parser.findall(text))
# spans = [_.span for _ in matches]
# show_markup(text, spans)
# for match in parser.findall(text):
#     print (match.fact)

In [118]:
for index, row in subjects.iterrows():
    tx = row['subject']
    if (tx is not None) or (tx != 'NaN') or (str(tx) != 'nan'):
        for match in parser.findall(str(tx)):
                category = match.fact
                if category is not None:
                    subjects.at[index, 'category'] = str(category.name)
                else:
                    subjects.at[index, 'category'] = 'Nan'
    else:
        subjects.at[index, 'category'] = 'Nan'
